In [ ]:
!pip install openai==1.93.0

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from the specified path
load_dotenv(dotenv_path='/home/sparky/.dbt/.env')

In [ ]:
import polars as pl
from datetime import datetime
from openai import OpenAI
import pickle
import numpy as np
from tqdm import tqdm
import concurrent.futures
import time
import os
import json

## Read Raw Data

In [ ]:
raw_path = '/opt/spark/work-dir/data/exists_data/raw_embedding_demo.csv'

raw_df = pl.read_csv(source=raw_path, has_header=True)
raw_df.head()

In [ ]:
em_01 = raw_df.filter(pl.col("ecommerce_name") == 'ecommerce_01')
em_02 = raw_df.filter(pl.col("ecommerce_name") == 'ecommerce_02')

## Setup & Helper Function

In [ ]:
# Ensure OPENAI_API_KEY environment variable is set
client = OpenAI()

In [ ]:
# Define function: generate text embeddings
def generate_embeddings(text):
    """
    Generate embeddings for input text using OpenAI's text-embedding-3-small model.

    Args:
        text (str or list[str]): The text or list of texts to embed.

    Returns:
        list or list[list]: A list of embeddings, or a list of lists of embeddings if input was a list.
    """
    if not text:
        return []
    
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    embeddings = [record.embedding for record in response.data]
    return embeddings if isinstance(text, list) else embeddings[0]

In [ ]:
# Set embedding cache path
embedding_cache_path = "embeddings_cache.pkl"

# Try to load cached embeddings
try:
    with open(embedding_cache_path, 'rb') as f:
        embedding_cache = pickle.load(f)
except FileNotFoundError:
    embedding_cache = {}

In [ ]:
# Define function: obtain text embeddings through caching mechanism
def embedding_from_string(string: str, embedding_cache=embedding_cache) -> list:
    """
    Get embedding for given text, using cache mechanism to avoid recomputation.

    Args:
        string (str): The input text to get the embedding for.
        embedding_cache (dict): A dictionary used as a cache for embeddings.

    Returns:
        list: The embedding vector for the input string.
    """
    if string not in embedding_cache.keys():
        embedding_cache[string] = generate_embeddings(string)
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
    return embedding_cache[string]


## Embed base product name

In [ ]:
# Define a dynamic batch size. You can adjust this value based on API limits and performance.
BATCH_SIZE = 60

print("Preparing base by generating embeddings for all base products...")
# Get all product names from the base dataframe
base_names = em_02.select('product_name').to_series().to_list()
base_prices = em_02.select('sale_price').to_series().to_list()

# 1. Identify which product names are new and need to be embedded.
names_to_embed = [name for name in base_names if name not in embedding_cache]

In [ ]:
# 2. If there are any new names, process them in manageable batches.
if names_to_embed:
    total_batches = -(-len(names_to_embed) // BATCH_SIZE)  # Ceiling division to calculate total batches
    print(f"Found {len(names_to_embed)} new products to embed. Processing in {total_batches} batches of up to {BATCH_SIZE} items each...")
    
    # Iterate through the new names in chunks of BATCH_SIZE
    for i in range(0, len(names_to_embed), BATCH_SIZE):
        current_batch_num = (i // BATCH_SIZE) + 1
        
        # Define the current batch of names
        batch_names = names_to_embed[i:i + BATCH_SIZE]
        print(f"  - Processing batch {current_batch_num}/{total_batches} ({len(batch_names)} items)...")
        
        # Generate embeddings for the current batch
        batch_embeddings = generate_embeddings(batch_names)
        
        # 3. Update the cache with the newly generated embeddings from this batch
        for name, embedding in zip(batch_names, batch_embeddings):
            embedding_cache[name] = embedding

        # 4. Save the updated cache to disk after each batch. This adds robustness.
        #    If the script fails, progress from completed batches is not lost.
        with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)
        print(f"  - Batch {current_batch_num} processed and cache saved.")
            
    print("All new embeddings have been generated and cached.")

# 5. Retrieve all embeddings from the cache to ensure the list is complete and in the correct order.
base_embeddings = [embedding_cache[name] for name in base_names]

In [ ]:
data_for_df = {
    'product_name': base_names,
    'embedding': base_embeddings,
    'sale_price': base_prices
}
product_embeddings_df = pl.DataFrame(data_for_df)
product_embeddings_df.head()

## Find Best Match

In [ ]:
import google.generativeai as genai
from google.genai import types

In [ ]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
gemini_client = genai.Client()

In [ ]:
# --- 1. CORE MATCHING LOGIC (Unchanged) ---
def find_best_match_in_context(query_name, context_embeddings_np):
    """
    Finds the single most similar product in the context catalog for a given query name.
    """
    # The embedding for the query product is generated here, on-the-fly.
    query_embedding = embedding_from_string(query_name)
    similarities = np.dot(context_embeddings_np, query_embedding) / \
                   (np.linalg.norm(context_embeddings_np, axis=1) * np.linalg.norm(query_embedding))
    best_index = np.argmax(similarities)
    return best_index, similarities[best_index]

In [ ]:
# --- 2. NEW GEMINI-BASED VERIFICATION LOGIC (Unchanged) ---
def verify_matches_with_gemini(product_pairs: list[dict]):
    """
    Sends a list of product pairs to the Gemini API for verification.

    Args:
        product_pairs (list[dict]): A list of dictionaries, where each dict contains
                                    'query_product_name' and 'candidate_product_name'.

    Returns:
        list: A list of verification results from the LLM.
    """
    if not product_pairs:
        return []

    # Construct the prompt with up to 10 pairs
    prompt_lines = [
        "You are a precise product matching assistant. For each numbered item below, determine if the 'Base Product' and the 'Candidate Product' are the same item.",
        "Your response must be a valid JSON object containing a single key 'verifications', which holds a list of JSON objects.",
        "Each object in the list must correspond to an item in the input and contain a single key: 'match_found' with a boolean value (true or false).",
        "Ensure the order of your response list matches the order of the products provided.",
        "\n--- PRODUCT PAIRS ---"
    ]

    for i, pair in enumerate(product_pairs, 1):
        prompt_lines.append(f"\n{i}. Base Product: '{pair['query_product_name']}'")
        prompt_lines.append(f"   Candidate Product: '{pair['candidate_product_name']}'")

    prompt = "\n".join(prompt_lines)

    try:
        # 3. Call the generate_content method with the prompt.
        print(f"\nSending {len(product_pairs)} pairs to Gemini for verification...")
        response = gemini_client.models.generate_content(
            model='gemini-2.5-flash-lite-preview-06-17',
            contents=prompt,
            config={
                "response_mime_type":"application/json"
            }
        )
        
        # --- FIXED SECTION END ---
        
        # The response.text already contains the JSON string
        result_json = json.loads(response.text)
        return result_json.get('verifications', [])

    except Exception as e:
        print(f"An error occurred while calling the Gemini API: {e}")
        # Return a list of 'False' results to prevent crashing the main loop
        return [{'match_found': False} for _ in product_pairs]

In [ ]:
# --- 3. MODIFIED MAIN PROCESSING SCRIPT (Unchanged) ---

# Define the batch size for each API call
API_CALL_SIZE = 10

# df_query corresponds to the new products to be checked.
# df_base corresponds to the existing product catalog.
df_query = em_01
df_base = product_embeddings_df
base_embeddings_np = np.array(df_base['embedding'].to_list())

results_list = []
# This correctly iterates through your 'em_01' DataFrame row by row.
query_rows = list(df_query.iter_rows(named=True))

In [ ]:
results_list = []
query_rows = list(df_query.iter_rows(named=True))

print(f"Starting product matching in sequential batches of {BATCH_SIZE}...")

In [ ]:
print(f"Starting product matching in chunks of {API_CALL_SIZE}...")

# Process the query dataframe in sequential chunks
for i in range(0, len(query_rows), API_CALL_SIZE):
    product_chunk = query_rows[i:i + API_CALL_SIZE]
    chunk_num = (i // API_CALL_SIZE) + 1
    print(f"\n--- Processing Chunk {chunk_num} ({len(product_chunk)} products) ---")
    
    chunk_data_for_verification = []
    chunk_original_data = []

    for row in product_chunk:
        query_product_name = row.get('product_name')
        if not query_product_name:
            continue

        # Find the best candidate match from the base catalog
        match_index, similarity = find_best_match_in_context(query_product_name, base_embeddings_np)
        matched_product_info = df_base.row(match_index, named=True)
        
        chunk_data_for_verification.append({
            'query_product_name': query_product_name,
            'candidate_product_name': matched_product_info['product_name']
        })
        
        chunk_original_data.append({
            'original_row': row,
            'match_info': matched_product_info,
            'similarity': similarity
        })

    # Call the Gemini API for the entire chunk
    llm_results = verify_matches_with_gemini(chunk_data_for_verification)

    if len(llm_results) != len(chunk_original_data):
        print(f"Warning: Mismatch between sent items ({len(chunk_original_data)}) and received results ({len(llm_results)}). Skipping chunk.")
        continue

    for original_data, llm_result in zip(chunk_original_data, llm_results):
        try:
            is_match = llm_result.get('match_found', False)
        except (AttributeError, TypeError):
            print(f"Could not parse LLM result: {llm_result}. Treating as no match.")
            is_match = False

        query_row = original_data['original_row']
        match_info = original_data['match_info']
        similarity_score = original_data['similarity']

        results_list.append({
            'base_product_name (from line)': query_row['product_name'],
            'base_product_name (from watson)': match_info['product_name'],
            'price_from_line': query_row['sale_price'],
            'price_from_watson': match_info['sale_price'],
            'similarity': float(similarity_score),
            'verified_match': is_match
        })
    
    print(f"Finished processing Chunk {chunk_num}.")


print("\n--- All chunks processed. Final results compiled. ---")

In [ ]:
# --- FINAL STEP: CONVERT RESULTS TO DATAFRAME AND SAVE TO CSV (Unchanged) ---
if results_list:
    print("Converting results to a Polars DataFrame...")
    results_df = pl.DataFrame(results_list)

    output_filename = "product_matching_results_gemini.csv"
    try:
        results_df.write_csv(output_filename)
        print(f"\nSuccessfully saved {len(results_df)} results to '{output_filename}'")
        print("Final DataFrame head:")
        print(results_df.head())
    except Exception as e:
        print(f"Error saving results to CSV: {e}")
else:
    print("No results were generated to save.")
